-Getting player data since 2000 from Season Stats CSV
-Getting team roster 

In [94]:
#Libraries/Imports

import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [201]:
allLinks = []

for year in range(2000,2022):
    url = "https://www.basketball-reference.com/leagues/NBA_{}.html".format(year)

    response = requests.request("GET", url).text
    soupStruct = bs(response)

    allTables = soupStruct.findAll('table')[:2]
    
    for table in allTables:
        for row in table.find_all('tr'):
            for th in row.find_all('th'):
                aTag = th.a
                if aTag != None:
                    allLinks.append(aTag['href'])                      

In [202]:
rosterData = pd.DataFrame(data = None, index = None, columns = None, dtype = None, copy = None)

for link in allLinks:
    #creating the link with array with combo of links and years
    url = "https://www.basketball-reference.com" + link
    
    #sending request
    response = requests.request("GET", url).text
    soup = bs(response)
    
    #finding the record by looking for the word "Record:"
    record = soup.find(lambda tag: tag.name == "strong" and "Record:" in tag.text).next_sibling 
    winsLoss = record.split(',')[0].strip().split('-') #looking for wins
    wins = int(winsLoss[0])
    losses = int(winsLoss[1])
    
    #finding the correct table from the team website by looking for keywords "Totals"
    rosterTable = soup.find(lambda tag: tag.name == "h2" and "Totals" in tag.text).find_next("table")
    
    #converting from html to dataframe
    teamRosterDF = pd.read_html(str(rosterTable))[0]
    
    #adding the year,team, wins, and losses to the dataframe
    teamRosterDF["Year"] = url[-9:-5]
    teamRosterDF["Team"] = url[-13:-10]
    teamRosterDF["Wins"] = wins
    teamRosterDF["Losses"] = losses
    teamRosterDF.rename(columns = {'Unnamed: 1':'Name'}, inplace = True) #renaming the name column
    rosterData = rosterData.append(teamRosterDF) #adding to the complete roster data
 
rosterData

,Rk,Name,Age,G,GS,MP,FG,FGA,FG%,3P,...,AST,STL,BLK,TOV,PF,PTS,Year,Team,Wins,Losses
0,1.0,Jamal Mashburn,27.0,76,76.0,2828,515,1158,0.445,112,...,298,79,14,180,215,1328,2000,MIA,52,30
1,2.0,Alonzo Mourning,29.0,79,78.0,2748,652,1184,0.551,0,...,123,40,294,217,308,1718,2000,MIA,52,30
2,3.0,Dan Majerle,34.0,69,69.0,2308,170,422,0.403,110,...,206,89,17,62,156,506,2000,MIA,52,30
3,4.0,P.J. Brown,30.0,80,80.0,2302,322,671,0.480,0,...,145,65,61,100,264,764,2000,MIA,52,30
4,5.0,Anthony Carter,24.0,79,30.0,1859,201,509,0.395,3,...,378,93,5,173,167,498,2000,MIA,52,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26,27.0,Bruno Caboclo,25.0,6,0.0,36,8,17,0.471,0,...,1,0,2,4,6,17,2021,HOU,17,55
27,28.0,Cameron Reynolds,25.0,2,0.0,34,5,16,0.313,3,...,2,0,0,0,2,13,2021,HOU,17,55
28,29.0,Brodric Thomas,24.0,4,0.0,24,2,7,0.286,1,...,4,1,1,4,5,10,2021,HOU,17,55
29,30.0,Ray Spalding,23.0,2,0.0,19,2,4,0.500,0,...,0,0,2,3,3,4,2021,HOU,17,55


In [203]:
csv = rosterDataNew.to_csv("all_player_data.csv")